In [126]:
from datetime import datetime, timedelta
import os
import re
import pandas as pd
import numpy as np

from calculate_flux import get_fluency_flux
from calculate_flux import read_count_file
from calculate_flux import calculate_open_periods


# Directory path
directory = "./"
distance_factor_file = "distance_file.txt"
allowed_devices = ['bruno1', 'bruno2']
shutter_file = "shutter.txt"

# Time window for each run
time_unit_offset = timedelta(hours=0.1)


In [127]:
# Get all countlog files in the directory
countlog_files = [filename for filename in os.listdir(directory) if filename.startswith("countlog")]

# Sort the countlog files in chronological order
countlog_files.sort(key=lambda x: datetime.strptime(re.search(r"\d{4}-\d{2}-\d{2}", x).group(), "%Y-%m-%d"))

# Initialize an empty dataframe
neutron_count_df = pd.DataFrame()

# Loop through the countlog files in chronological order
for filename in countlog_files:
    # Get the full file path
    file_path = os.path.join(directory, filename)

    print(filename)
    # Read the count file and append it to the dataframe
    neutron_count = read_count_file(file_path)
    neutron_count_df = pd.concat([neutron_count_df, pd.DataFrame(neutron_count)]).reset_index(drop=True)


neutron_count_df = neutron_count_df.rename(columns={0: "Timestamp", 1: "Fission Count"})

# Load all distances before continue
distance_data = pd.read_csv(distance_factor_file, sep="\t")

countlog-2023-12-04.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-05.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-06.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-07.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-08.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-09.txt
Ignoring line (malformed):Data From ChipIR

countlog-2023-12-10.txt
Ignoring line (malformed):Data From ChipIR



In [128]:
#Filter out the data that is not in the distance file for each device
devices_dataframe = {}

for device in distance_data["board"].unique():
    if device in allowed_devices:
        distance_line = distance_data[distance_data["board"] == device]
        start_line = distance_line["start"].iloc[0]
        end_line = distance_line["end"].iloc[0]

        start_dt = pd.to_datetime(start_line, format='%d/%m/%Y %H:%M:%S')
        end_dt = pd.to_datetime(end_line, format='%d/%m/%Y %H:%M:%S')
        
        filtered_df = neutron_count_df[
            (neutron_count_df["Timestamp"] >= start_dt) &
            (neutron_count_df["Timestamp"] <= end_dt)
        ]
        
        devices_dataframe[device] = filtered_df

In [129]:
#Filter dataframes to remove open shutter times

# def display_periods(periods):
#     """ Display the open periods """
#     total_duration = timedelta()
#     for start, end in periods:
#         total_duration += end - start
        
#     total_duration_hours = total_duration.total_seconds() / 3600
#     print(f"Shutter open for a total time of {total_duration_hours} hours")

# periods = calculate_open_periods(shutter_file)
# display_periods(periods)

# shutter_dataframe = {}

# for device, df in devices_dataframe.items():
#     filtered_df = pd.DataFrame()
#     for start, end in periods:
#         filtered_df = pd.concat([filtered_df, df[(df["Timestamp"] >= start) & (df["Timestamp"] <= end)]])
#     shutter_dataframe[device] = filtered_df


In [130]:

device_dict = {}

for device in allowed_devices:
    device_flunce_pd = pd.DataFrame(columns=['Start Date', 'Flux', 'Time Beam Off', 'Fluence'])
    
    df = devices_dataframe[device]

    distance_line = distance_data[
        (distance_data["board"].str.contains(device))
    ]
    facility_factor = float(distance_line["facility_factor"])
    distance_attenuation = float(distance_line["Distance attenuation"])

    start_dt = df["Timestamp"].iloc[0]

    while start_dt <= df["Timestamp"].iloc[-1]:
        end_dt = start_dt + time_unit_offset
        
        if end_dt > df["Timestamp"].iloc[-1]:
            break

        mistake_time = pd.to_datetime("2023-12-08 10:23:00")
        if start_dt <= mistake_time and end_dt >= mistake_time:
            # print("aaaaaa",start_dt, end_dt,mistake_time)
            start_dt = start_dt + time_unit_offset
            continue

        neutron_count = df[(df["Timestamp"] >= start_dt) & (df["Timestamp"] <= end_dt)].values
        if len(neutron_count) == 0:
            start_dt = start_dt + time_unit_offset
            continue
        

        flux, time_beam_off = get_fluency_flux(
            start_dt=start_dt,
            end_dt=end_dt,
            neutron_count=neutron_count,
            facility_factor=facility_factor,
            distance_attenuation=distance_attenuation,
        )
        
        fluence = flux * time_unit_offset.total_seconds()  
        data = [
            [start_dt, flux, time_beam_off, flux * time_unit_offset.total_seconds()]
        ]
        device_flunce_pd = pd.concat([device_flunce_pd, pd.DataFrame(data, columns=['Start Date', 'Flux', 'Time Beam Off', 'Fluence'])], ignore_index=True)
        # print(start_dt, flux, time_beam_off, fluence)

        start_dt = end_dt
    
    device_dict[device] = device_flunce_pd



In [131]:
# Set display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

#Filter dataframes to remove open shutter times

def display_periods(periods):
    """ Display the open periods """
    total_duration = timedelta()
    for start, end in periods:
        total_duration += end - start
        
    total_duration_hours = total_duration.total_seconds() / 3600
    print(f"Shutter open for a total time of {total_duration_hours} hours")

periods = calculate_open_periods(shutter_file)
display_periods(periods)

# FILEPATH: /home/beforlin/projects/radhelper-embedded/post_processing/flux.ipynb
device_dict["bruno1"] = device_dict["bruno1"][device_dict["bruno1"]["Flux"] != 0]
total_time_beam_off = device_dict["bruno1"]["Time Beam Off"].sum()
num_time_windows = device_dict["bruno1"].shape[0] * time_unit_offset.total_seconds() / 3600
total_fluence = device_dict["bruno1"]["Fluence"].sum()
print(f"{round(total_time_beam_off/3600, 2)} hours of beam off out of {num_time_windows} hours") # in hours
print(f"Total Fluence = {total_fluence:.2e}")
# device_dict["bruno1"]

# 10:35:17.102 -- 14:07:12.658

Shutter open for a total time of 64.46666666666667 hours
3.72 hours of beam off out of 54.3 hours
Total Fluence = 8.99e+11


In [132]:
filtered_df = device_dict["bruno1"][device_dict["bruno1"]['Time Beam Off'] < 100]
 
filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 7]
filtered_df = filtered_df[filtered_df['Start Date'].dt.hour >=10]
filtered_df = filtered_df[filtered_df['Start Date'].dt.hour < 15]
# filtered_df = filtered_df[filtered_df['Start Date'].dt.minute >= 0]
# filtered_df = filtered_df[filtered_df['Start Date'].dt.minute <=58]
filtered_df


,Start Date,Flux,Time Beam Off,Fluence
374,2023-12-07 10:24:00.403,5.034025e+06,0.000,1.812249e+09
375,2023-12-07 10:30:00.403,4.998940e+06,0.000,1.799618e+09
376,2023-12-07 10:36:00.403,5.056479e+06,0.000,1.820333e+09
377,2023-12-07 10:42:00.403,5.026306e+06,0.000,1.809470e+09
378,2023-12-07 10:48:00.403,5.063496e+06,0.000,1.822859e+09
379,2023-12-07 10:54:00.403,5.009465e+06,0.000,1.803407e+09
380,2023-12-07 11:00:00.403,5.029815e+06,0.000,1.810733e+09
381,2023-12-07 11:06:00.403,4.978590e+06,0.000,1.792292e+09
382,2023-12-07 11:12:00.403,5.041042e+06,0.000,1.814775e+09
383,2023-12-07 11:18:00.403,5.013675e+06,0.000,1.804923e+09


In [133]:
from datetime import datetime, date, time
from datetime import timedelta

def largest_stretch(df):
    # Find the largest continuous stretch of rows without discontinuity
    max_stretch = 0
    current_stretch = 0
    start_index = 0
    end_index = 0

    for i in range(len(df) - 1):
        if df.index[i] + 1 == df.index[i + 1]:
            current_stretch += 1
        else:
            if current_stretch > max_stretch:
                max_stretch = current_stretch
                start_index = i - current_stretch
                end_index = i
            current_stretch = 0

    if current_stretch > max_stretch:
        max_stretch = current_stretch
        start_index = len(filtered_df) - current_stretch
        end_index = len(filtered_df) - 1

    return df.iloc[start_index:end_index + 1]



# Bruno1 config 2


In [134]:
filtered_df = device_dict["bruno1"][device_dict["bruno1"]['Time Beam Off'] < 100]
 
filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 7]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time > time(10,35)]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time < time(14,7)]

filter_stretch = largest_stretch(filtered_df)

# largest_stretch
end_date = filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)
print(f"Window : {filter_stretch['Start Date'].iloc[0]} -- {filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)}")
mean_fluence = filter_stretch['Fluence'].sum()
print(f"Mean Fluence: {mean_fluence:.2e}")

Window : 2023-12-07 10:36:00.403000 -- 2023-12-07 13:06:00.403000
Mean Fluence: 4.52e+10


In [135]:
filtered_df = device_dict["bruno1"][device_dict["bruno1"]['Time Beam Off'] < 100]
 
filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 7]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time > time(14,8)]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time < time(18,30)]

filter_stretch = largest_stretch(filtered_df)

# largest_stretch
end_date = filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)
print(f"Window : {filter_stretch['Start Date'].iloc[0]} -- {filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)}")
mean_fluence = filter_stretch['Fluence'].sum()
print(f"Mean Fluence: {mean_fluence:.2e}")

Window : 2023-12-07 16:36:00.403000 -- 2023-12-07 18:30:00.403000
Mean Fluence: 3.43e+10


In [136]:
filtered_df = device_dict["bruno1"][device_dict["bruno1"]['Time Beam Off'] < 100]
 
filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 7]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time > time(18,31)]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time < time(19,20)]

filter_stretch = largest_stretch(filtered_df)

# largest_stretch
end_date = filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)
print(f"Window : {filter_stretch['Start Date'].iloc[0]} -- {filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)}")
mean_fluence = filter_stretch['Fluence'].sum()
print(f"Mean Fluence: {mean_fluence:.2e}")

Window : 2023-12-07 18:42:00.403000 -- 2023-12-07 19:24:00.403000
Mean Fluence: 1.27e+10


In [137]:
filtered_df = device_dict["bruno1"][device_dict["bruno1"]['Time Beam Off'] < 100]
 
filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 7]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time > time(19,21)]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time < time(23,23)]

filter_stretch = largest_stretch(filtered_df)

# largest_stretch
end_date = filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)
print(f"Window : {filter_stretch['Start Date'].iloc[0]} -- {filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)}")
mean_fluence = filter_stretch['Fluence'].sum()
print(f"Mean Fluence: {mean_fluence:.2e}")

Window : 2023-12-07 19:24:00.403000 -- 2023-12-07 21:54:00.403000
Mean Fluence: 4.48e+10


In [138]:
filtered_df = device_dict["bruno1"][device_dict["bruno1"]['Time Beam Off'] < 100]
 
filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 8]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time > time(10,43)]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time < time(17,7)]

filter_stretch = largest_stretch(filtered_df)

# largest_stretch
end_date = filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)
print(f"Window : {filter_stretch['Start Date'].iloc[0]} -- {filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)}")
mean_fluence = filter_stretch['Fluence'].sum()
print(f"Mean Fluence: {mean_fluence:.2e}")



Window : 2023-12-08 15:12:00.403000 -- 2023-12-08 17:06:00.403000
Mean Fluence: 3.33e+10


# Bruno 1 Config 1


In [139]:
filtered_df = device_dict["bruno1"][device_dict["bruno1"]['Time Beam Off'] < 100]
 
filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 6]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time > time(17,40)]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time < time(22,55)]

filter_stretch = largest_stretch(filtered_df)

# largest_stretch
end_date = filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)
print(f"Window : {filter_stretch['Start Date'].iloc[0]} -- {filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)}")
mean_fluence = filter_stretch['Fluence'].sum()
print(f"Mean Fluence: {mean_fluence:.2e}")

Window : 2023-12-06 18:06:00.403000 -- 2023-12-06 18:54:00.403000
Mean Fluence: 1.41e+10


In [141]:
filtered_df = device_dict["bruno1"][device_dict["bruno1"]['Time Beam Off'] < 100]
 
# filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 6]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time > time(22,56)]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time < time(2,17)]

filter_stretch = largest_stretch(filtered_df)
filter_stretch
# largest_stretch
# end_date = filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)
# print(f"Window : {filter_stretch['Start Date'].iloc[0]} -- {filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)}")
# mean_fluence = filter_stretch['Fluence'].sum()
# print(f"Mean Fluence: {mean_fluence:.2e}")

,Start Date,Flux,Time Beam Off,Fluence


In [142]:
filtered_df = device_dict["bruno1"][device_dict["bruno1"]['Time Beam Off'] < 100]
 
filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 7]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time > time(2,18)]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time < time(5,39)]

filter_stretch = largest_stretch(filtered_df)
filter_stretch
# largest_stretch
end_date = filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)
print(f"Window : {filter_stretch['Start Date'].iloc[0]} -- {filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)}")
mean_fluence = filter_stretch['Fluence'].sum()
print(f"Mean Fluence: {mean_fluence:.2e}")

Window : 2023-12-07 02:18:00.403000 -- 2023-12-07 04:36:00.403000
Mean Fluence: 4.18e+10


In [143]:
filtered_df = device_dict["bruno1"][device_dict["bruno1"]['Time Beam Off'] < 100]
 
filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 7]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time > time(5,40)]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time < time(8,58)]

filter_stretch = largest_stretch(filtered_df)
filter_stretch
# largest_stretch
end_date = filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)
print(f"Window : {filter_stretch['Start Date'].iloc[0]} -- {filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)}")
mean_fluence = filter_stretch['Fluence'].sum()
print(f"Mean Fluence: {mean_fluence:.2e}")

Window : 2023-12-07 05:42:00.403000 -- 2023-12-07 07:36:00.403000
Mean Fluence: 3.41e+10


In [144]:
filtered_df = device_dict["bruno1"][device_dict["bruno1"]['Time Beam Off'] < 100]
 
filtered_df = filtered_df[filtered_df['Start Date'].dt.day == 7]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time > time(8,58)]
filtered_df = filtered_df[filtered_df['Start Date'].dt.time < time(10,3)]

filter_stretch = largest_stretch(filtered_df)
filter_stretch
# largest_stretch
# end_date = filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)
# print(f"Window : {filter_stretch['Start Date'].iloc[0]} -- {filter_stretch['Start Date'].iloc[-1] + timedelta(minutes=6)}")
# mean_fluence = filter_stretch['Fluence'].sum()
# print(f"Mean Fluence: {mean_fluence:.2e}")

IndexError: single positional indexer is out-of-bounds